In [1]:
import dask.dataframe as dd
import numpy as np
import os
import pandas as pd

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

#### Thay đổi hiển thị ####
pd.reset_option('display.max_columns')
pd.reset_option('display.max_rows')
pd.set_option('display.max_row', None)
pd.set_option('display.max_columns', None)

config = ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.9  # Adjust as needed
session = InteractiveSession(config=config)
#### Change display ####

# input_file = ["/mnt/c/Users/hoang/FileCSV_DACN_2025/parquet_shuffled_IoT23", "C:\\Users\\hoang\\FileCSV_DACN_2025\\parquet_shuffled_IoT23"]
input_file = ["/mnt/c/Users/hoang/FileCSV_DACN_2025/medbiot.csv", "C:\\Users\\hoang\\FileCSV_DACN_2025\\medbiot.csv"]


if os.name == 'nt':
    input_file = input_file[1]
else:
    input_file = input_file[0]

# dictTypes = {}
# df = dd.read_parquet(input_file)
# for col in df.columns:
#     if col.startswith('proto') == True:
#         dictTypes[col] = 'int32'
#     elif col.startswith('service_') == True:
#         dictTypes[col] = 'int32'
#     # elif col == 'label':
#     #     dictTypes[col]= 'int32'
#     elif col.startswith('detailed-label'):
#         dictTypes[col] = 'str'
#     else:
#         dictTypes[col]='float32'

df = dd.read_csv(input_file) # dtype = dictTypes

label_map = {'benign': 0, 'torii': 1, 'mirai_spread': 2, 'bashlite_spread': 3, 'mirai_c2': 4, 'bashlite_c2': 5}
df['label'] = df['label'].map(label_map).astype('int32')
# phân loại đa nhãn, k dùng
# df = df.replace(r'[N|n][a|A][N|n]', 0)
# df = df.replace(np.nan, 0)
print(df.columns)
# print(df['duration'].value_counts().compute())
print(df.tail(50))
print(df.head(50))

c:\Users\hoang\AppData\Local\Programs\Python\Python311\Lib\site-packages\dask\dataframe\dask_expr\_collection.py:4208: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('label', 'float64'))

  warnings.warn(meta_warning(meta))


Index(['MI_dir_5_weight', 'MI_dir_5_mean', 'MI_dir_5_std', 'MI_dir_3_weight',
       'MI_dir_3_mean', 'MI_dir_3_std', 'MI_dir_1_weight', 'MI_dir_1_mean',
       'MI_dir_1_std', 'MI_dir_0.1_weight',
       ...
       'HpHp_0.1_covariance_0_1', 'HpHp_0.1_pcc_0_1', 'HpHp_0.01_weight_0',
       'HpHp_0.01_mean_0', 'HpHp_0.01_std_0', 'HpHp_0.01_radius_0_1',
       'HpHp_0.01_magnitude_0_1', 'HpHp_0.01_covariance_0_1',
       'HpHp_0.01_pcc_0_1', 'label'],
      dtype='object', length=101)
       MI_dir_5_weight  MI_dir_5_mean  MI_dir_5_std  MI_dir_3_weight  \
39639         1.000000      60.000000  0.000000e+00         1.000000   
39640         1.020819      73.714476  3.915809e+00         1.097967   
39641         1.000000      60.000000  0.000000e+00         1.000000   
39642         2.017386      66.916334  4.899300e+01         2.095750   
39643         3.014479      74.242199  1.408541e+02         3.093937   
39644         1.992549      60.000000  9.094947e-13         1.995523   
39645  

In [ ]:
import numpy as np
import pandas as pd
import dask.dataframe as dk
import tensorflow as tf 
from tensorflow.keras.utils import Sequence, to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from datetime import datetime, timedelta
from tensorflow import keras

#Global var 
batch_size = 512
ratio_test_all = 0.2

from dask_ml.model_selection import train_test_split 
# Bước 1: Tách 80% train, 20% còn lại (val + test)
train_df, val_test_df = train_test_split(df, test_size=0.20, random_state=42, shuffle=True)
val_df, test_df = train_test_split(val_test_df, test_size=0.75, random_state=42, shuffle=True)
def val_test_df


In [ ]:
features_len = len(df.columns)-1
print("Feature Len: ",features_len)
batchSize = 512
def dask_to_tf_dataset(dask_df, num_classes):
    def generator():
        # for partition in dask_df.partitions:
        #     batch = partition.compute() 
        for batch in dask_df.to_delayed():
            batch=batch.compute()  
            if batch.empty:
                continue
            
            X = batch.drop(columns='label').values.astype(np.float32)
            y = batch['label'].values.astype(np.int32)
            y_onehot = to_categorical(y, num_classes=num_classes)

            num_splits = max(1, len(X)//batchSize) # Đảm bảo không chia nhỏ quá mức
            X_batches = np.array_split(X, num_splits)
            y_batches = np.array_split(y_onehot, num_splits) # y_onehot

            for X_batch, y_batch in zip(X_batches, y_batches):
                yield X_batch, y_batch
                
    output_signature = ( 
        tf.TensorSpec(shape=(None, features_len), dtype=tf.float32), 
        tf.TensorSpec(shape=(None, num_classes), dtype=tf.int32), # 3
    )
    
    return tf.data.Dataset.from_generator(generator, output_signature=output_signature).prefetch(tf.data.AUTOTUNE)

Feature Len:  100


57.083984375
57.119140625
56.77734375
57.6953125


In [36]:
train_gen = dask_to_tf_dataset(train_df,  6).repeat()
val_gen = dask_to_tf_dataset(val_df, 6).repeat()
test_gen = dask_to_tf_dataset(test_df, 6)

In [41]:
import math
batchSize=512
n_samples = np.ceil(train_df.shape[0])
steps_per_epoch = int(n_samples / (batchSize))
validation_steps =int (steps_per_epoch/ 16)
print("Steps/Epoch: ", steps_per_epoch)

Steps/Epoch:  18825


In [ ]:
########### Enable XLA ###############
# tf.config.optimizer.set_jit(True)

########### Nếu không dùng XLA ###########
# import os
# os.environ["TF_XLA_FLAGS"] = "--tf_xla_auto_jit=0"

import gc
gc.collect()
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# shape
features, labels = next(iter(train_gen))
input_shape = (features.shape[1], 1)
output_shape = labels.shape[1]

print(f"Input Shape: {input_shape} \n Output Shape: {output_shape}")

# Định nghĩa mô hình CNN
# VGG, ...
# Conv2D, tabular, ...
# HE, tính tương thích của HE với CNN
# Tính chất data in, out; Học tăng cường
start_time = datetime.now()

# model = keras.Sequential([
#     layers.Input(shape=input_shape),
#     layers.Conv1D(filters=128, kernel_size=3, padding="same", activation="relu"),
#     layers.BatchNormalization(),
#     layers.Conv1D(filters=128, kernel_size=3,  padding="same",activation="relu"),
#     layers.BatchNormalization(),
#     layers.MaxPooling1D(pool_size=2),
#     layers.Dropout(0.1),
#     layers.Flatten(),
#     layers.Dense(128, activation='relu'),
#     layers.BatchNormalization(),
#     layers.Dense(1, activation='sigmoid') #softmax
# ])
# adam_optimizer = optimizers.Adam(learning_rate=0.0001)
# model.compile(optimizer=adam_optimizer, loss='binary_crossentropy', metrics=['accuracy'])
# model.summary()

model = keras.Sequential([
    layers.Input(shape=input_shape),
    layers.Conv1D(filters=128, kernel_size=3, padding="same", activation="relu"),
    layers.BatchNormalization(),
    # layers.Conv1D(filters=128, kernel_size=3,  padding="same",activation="relu"),
    # layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Dropout(0.3),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(output_shape, activation='softmax') #dtype='float32'
])
adam_optimizer = optimizers.Adam(learning_rate=0.002)
model.compile(optimizer=adam_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

#sparse khi không onehot
# for batch in dataloader:
#     X_batch = batch[:, :-1]
#     y_batch = batch[:, -1]
#     y_onehot = to_categorical(y_batch, num_classes=10)
    
#     model.train_on_batch(X_batch, y_onehot, verbose=1)
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping, ModelCheckpoint
csv_logger = CSVLogger("Centralized_Log/"+ datetime.now().strftime("Month%mDay%d__%Hh%Mp")+".csv" , append=True)

model_path =  "Centralized_Model/cnn_model_" + datetime.now().strftime("Month%mDay%d__%Hh%Mp")
print("Model Path: ", model_path)

callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True),
    ModelCheckpoint(model_path+"_best.keras", monitor='val_loss', save_best_only=True, verbose=1),
    csv_logger
]
# log_dir = "Tensorflow_log/profile"

# Log - tensorflow, bottle neck hay không
# tf.profiler.experimental.start(log_dir)
model.fit(train_gen, epochs=10, validation_data=val_gen, 
          validation_steps=validation_steps, steps_per_epoch= steps_per_epoch, verbose = 1, callbacks=callbacks)
# tf.profiler.experimental.stop()

end_time = datetime.now()
simulated_time = end_time - start_time

# Lưu mô hình
model.save(model_path+".keras")

print(f"Simulated time: {simulated_time}")

Input Shape: (100, 1) 
 Output Shape: 6


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_11 (Conv1D)              │ (None, 100, 128)       │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_22          │ (None, 100, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_11 (MaxPooling1D) │ (None, 50, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 50, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_11 (Flatten)            │ (None, 6400)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 128)            │       819,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_23          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 821,638 (3.13 MB)

 Trainable params: 821,126 (3.13 MB)

...

  968/18825 ━━━━━━━━━━━━━━━━━━━━ 20:07 68ms/step - accuracy: 0.1402 - loss: nan

E0000 00:00:1748254175.122319   87575 buffer_comparator.cc:156] Difference at 0: 0.0323181, expected -nan
E0000 00:00:1748254175.122423   87575 buffer_comparator.cc:156] Difference at 2: 0.041626, expected -nan
E0000 00:00:1748254175.122439   87575 buffer_comparator.cc:156] Difference at 4: 0.0429688, expected -nan
E0000 00:00:1748254175.122444   87575 buffer_comparator.cc:156] Difference at 6: 0.0248108, expected -nan
E0000 00:00:1748254175.122448   87575 buffer_comparator.cc:156] Difference at 8: 0.0596924, expected -nan
E0000 00:00:1748254175.122451   87575 buffer_comparator.cc:156] Difference at 10: 0.0303802, expected -nan
E0000 00:00:1748254175.122455   87575 buffer_comparator.cc:156] Difference at 12: 0.0427551, expected -nan
E0000 00:00:1748254175.122459   87575 buffer_comparator.cc:156] Difference at 14: 0.0111389, expected -nan
E0000 00:00:1748254175.122462   87575 buffer_comparator.cc:156] Difference at 16: 0.0837402, expected -nan
E0000 00:00:1748254175.122465   87575 buffe

 1555/18825 ━━━━━━━━━━━━━━━━━━━━ 17:50 62ms/step - accuracy: 0.1411 - loss: nan

KeyboardInterrupt: 

: 